In [6]:
from sparksampling import Submitter
from sparksampling.utilities.var import FILE_TYPE_CSV
from sparksampling.utilities.var import SIMPLE_RANDOM_SAMPLING_METHOD
submitter = Submitter()

In [8]:
submit_response = submitter.submit_sampling_simplejob('hdfs://localhost:9000/dataset/titanic/train.csv',
                                              method=SIMPLE_RANDOM_SAMPLING_METHOD,
                                              file_type=FILE_TYPE_CSV,
                                              fraction=0.9,
                                              with_header=True)
job_id = submit_response.data['job_id']
submit_response

request: http://localhost:8000/v1/sampling/simplejob/
2021-03-18 18:19:02,040 DEBUG connectionpool.py[231] Starting new HTTP connection (1): localhost:8000
2021-03-18 18:19:02,062 DEBUG connectionpool.py[461] http://localhost:8000 "POST /v1/sampling/simplejob/ HTTP/1.1" 200 49


In [9]:
submit_response.to_dict()

{'code': 0, 'msg': '', 'data': {'job_id': 10042}}

In [10]:
submitter.get_sampling_job_details(job_id).to_dict()

request: http://localhost:8000/v1/sampling/query/job/
2021-03-18 18:19:09,048 DEBUG connectionpool.py[231] Starting new HTTP connection (1): localhost:8000
2021-03-18 18:19:09,052 DEBUG connectionpool.py[461] http://localhost:8000 "POST /v1/sampling/query/job/ HTTP/1.1" 200 462


{'code': 0,
 'msg': '',
 'data': {'job_id': 10042,
  'job_status': 'Succeed',
  'msg': 'succeed',
  'method': 'Simple Random Sampling',
  'start_time': '2021/03/18/ 18:19:02',
  'end_time': '2021/03/18 18:19:03',
  'simpled_file_path': 'hdfs://localhost:9000/dataset/titanic/train.csv-sampled-1616062742.0438735',
  'request_data': "{'path': 'hdfs://localhost:9000/dataset/titanic/train.csv', 'method': 'random', 'type': 'csv', 'with_header': True, 'conf': {'fraction': 0.9}}"}}

In [11]:
data = submitter.get_statistics(job_id=job_id, from_sampling=True, file_type=FILE_TYPE_CSV,with_header=True).to_pandas()
data

request: http://localhost:8000/v1/evaluation/statistics/
2021-03-18 18:19:12,605 DEBUG connectionpool.py[231] Starting new HTTP connection (1): localhost:8000
2021-03-18 18:19:13,024 DEBUG connectionpool.py[461] http://localhost:8000 "POST /v1/evaluation/statistics/ HTTP/1.1" 200 1369


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
summary,,,,,,,,,,,,
count,800,800,800,800,800,644,800,800,800,800,182,798
mean,447.36625,0.38375,2.31625,None,None,29.899984472049685,0.5,0.3475,238760.35456110156,31.96637437499997,None,None
stddev,255.85095123430136,0.4866024197367053,0.8349295309498075,None,None,13.617861812410364,1.069546591968059,0.804706897876091,407001.6427253449,51.17323514604554,None,None
min,10,0,1,"""Andersson, Mr. August Edvard (""""Wennerstrom"""")""",female,0.42,0,0,110152,0,A10,C
max,99,1,3,"van Melkebeke, Mr. Philemon",male,9,8,6,WE/P 5735,93.5,T,S


In [12]:
evaluation_job = submitter.submit_evaluation_job(compare_job_id=job_id,file_type=FILE_TYPE_CSV)

request: http://localhost:8000/v1/evaluation/job/
2021-03-18 18:19:20,567 DEBUG connectionpool.py[231] Starting new HTTP connection (1): localhost:8000
2021-03-18 18:19:20,588 DEBUG connectionpool.py[461] http://localhost:8000 "POST /v1/evaluation/job/ HTTP/1.1" 200 49


In [13]:
evaluation_job_id = evaluation_job.data['job_id']
evaluation_job.to_dict()

{'code': 0, 'msg': '', 'data': {'job_id': 50013}}

In [18]:
evaluation_job_data = submitter.get_evaluation_job_details(job_id=evaluation_job_id)
evaluation_job_data.to_dict()

request: http://localhost:8000/v1/evaluation/query/job/
2021-03-18 18:21:09,414 DEBUG connectionpool.py[231] Starting new HTTP connection (1): localhost:8000
2021-03-18 18:21:09,418 DEBUG connectionpool.py[461] http://localhost:8000 "POST /v1/evaluation/query/job/ HTTP/1.1" 200 2565


{'code': 0,
 'msg': '',
 'data': {'job_id': 50013,
  'job_status': 'Succeed',
  'msg': 'succeed',
  'method': 'Compare Evaluation',
  'start_time': '2021/03/18/ 18:19:21',
  'end_time': '2021/03/18 18:19:21',
  'result': '{\'PassengerId\': {\'count\': \'800\', \'mean\': \'447.36625\', \'stddev\': \'255.85095123430136\', \'min\': \'10\', \'max\': \'99\', \'mean_bias\': 0.003063340807174842, \'stddev_bias\': 0.005839783735732211, \'score\': 0.9955484377285465}, \'Survived\': {\'count\': \'800\', \'mean\': \'0.38375\', \'stddev\': \'0.4866024197367053\', \'min\': \'0\', \'max\': \'1\', \'mean_bias\': 0.00023026315789472028, \'stddev_bias\': 2.0480119986323137e-05, \'score\': 0.9998746283610594}, \'Pclass\': {\'count\': \'800\', \'mean\': \'2.31625\', \'stddev\': \'0.8349295309498075\', \'min\': \'1\', \'max\': \'3\', \'mean_bias\': 0.0032954545454545905, \'stddev_bias\': 0.0013655654820842675, \'score\': 0.9976694899862305}, \'Name\': {\'count\': \'800\', \'mean\': None, \'stddev\': None,

In [19]:
evaluation_job_data.to_pandas()

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket
count,644,182,798,800,800,800,800,800,800,800,800,800
mean,29.899984472049685,None,None,31.96637437499997,None,0.3475,447.36625,2.31625,None,0.5,0.38375,238760.35456110156
stddev,13.617861812410364,None,None,51.17323514604554,None,0.804706897876091,255.85095123430136,0.8349295309498075,None,1.069546591968059,0.4866024197367053,407001.6427253449
min,0.42,A10,C,0,"""Andersson, Mr. August Edvard (""""Wennerstrom"""")""",0,10,1,female,0,0,110152
max,9,T,S,93.5,"van Melkebeke, Mr. Philemon",6,99,3,male,8,1,WE/P 5735
mean_bias,0.00676339,0,0,0.00738517,0,0.0893456,0.00306334,0.00329545,0,0.0439914,0.000230263,0.0828147
stddev_bias,0.0625502,0,0,0.0297787,0,0.00167522,0.00583978,0.00136557,0,0.0301039,2.04801e-05,0.136994
score,0.965343,0,0,0.981418,0,0.95449,0.995548,0.997669,0,0.962952,0.999875,0.890096
